<a href="https://colab.research.google.com/github/BanSangSu/Huggingface-AI-Agents-course/blob/main/unit2/with_langgraph/agent_with_search_and_weather_tools_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U langchain langchain_community langchain_openai langgraph langchain_core duckduckgo-search pyowm

## Access to the Web (search)

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()
results = search_tool("Who's the current President of Korea?")
print(results)

## Creating a Custom Tool for Weather Information to Schedule the Fireworks

In [ ]:
import os
from getpass import getpass

os.environ["OPENWEATHERMAP_API_KEY"] = getpass()

In [ ]:
from langchain.tools import Tool
from langchain_community.utilities import OpenWeatherMapAPIWrapper

def get_weather_info(location: str) -> str:
    """Fetches dummy weather information for a given location.
    You have to type city,Country Code . ex) Seoul,KR """
    weather = OpenWeatherMapAPIWrapper()

    weather_data = weather.run(location)

    return weather_data


weather_info_tool = Tool(
    name="get_weather_info",
    func=get_weather_info,
    description="Fetches dummy weather information for a given location. You have to type city,Country Code . ex) Seoul,KR "
)

## Creating a Hub Stats Tool for Influential AI Builders

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from langchain.tools import Tool
from huggingface_hub import list_models

def get_hub_stats(author: str) -> str:
    """Fetches the most downloaded model from a specific author on the Hugging Face Hub."""
    try:
        models = list(list_models(author=author, sort="downloads", direction=-1, limit=1))

        if models:
            model = models[0]
            return f"The most downloaded model by {author} is {model.id} with {model.downloads:,} downloads."
        else:
            return f"No models found for author {author}."
    except Exception as e:
        return f"Error fetching models for {author}: {str(e)}"


hub_stats_tool = Tool(
    name="get_hub_stats",
    func=get_hub_stats,
    description="Fetches the most downloaded model from a specific author on the Hugging Face Hub."
)

print(hub_stats_tool("facebook"))

## Integrating Tools with Alfred (LLM)

In [ ]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API Key:")
os.environ["OPENAI_API_BASE"] = getpass("OpenAI OPENAI BASE URL:")

In [ ]:
from getpass import getpass

model = getpass("Model:")

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model=model,
    temperature=0
)

tools = [search_tool, weather_info_tool, hub_stats_tool]
llm_with_tools = llm.bind_tools(tools)

### Generate the AgentState and Agent graph

In [ ]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage
from langgraph.prebuilt import ToolNode
from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]


def assistant(state: AgentState):
    return {
        "messages": llm_with_tools.invoke(state["messages"]),
    }

builder = StateGraph(AgentState)

builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "assistant")
builder.add_conditional_edges("assistant", tools_condition)
builder.add_edge("tools", "assistant")

alfred = builder.compile()

In [ ]:
messages = [HumanMessage(content="Who is Facebook and what's their most popular model?")]
response = alfred.invoke({"messages": messages})

In [ ]:
print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
messages = [HumanMessage(content="How is the weather in Seoul? And who's the current President of Korea?")]
# messages = [HumanMessage(content="Who's the current President of Korea?")]
response = alfred.invoke({"messages": messages})

In [ ]:
print("🎩 Alfred's Response:")
print(response['messages'][-1].content)

In [ ]:
messages = [HumanMessage(content="How is the weather in Seoul?")]
response = alfred.invoke({"messages": messages})

In [ ]:
print("🎩 Alfred's Response:")
print(response['messages'][-1].content)